# Targeted poisoning attack on MNIST dataset

## Import Libraries

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from matplotlib import pyplot as plt
import collections

tf.get_logger().setLevel('ERROR')

import random
from tqdm import tqdm
import copy

2023-06-08 11:46:24.392576: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 11:46:24.999356: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Parameters declaration

In [2]:
n_clients = 100
n_test_clients = 20

n_dataset_epochs = 5
n_train_epochs = 50
batch_size = 20

client_learning_rate = 0.02
server_learning_rate = 1


hidden_units = 256
dropout = 0.2

mal_users_percentage = 0.2
# todo: could also be a list of values
target_value = 3
poisoned_value = 8



## Dataset Loading and manipulation

In [3]:
emnist_train, emnist_test = tff.simulation.datasets.cifar100.load_data()

2023-06-08 11:46:28.450235: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 201 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1
2023-06-08 11:46:28.450710: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 11093 MB memory:  -> device: 1, name: NVIDIA TITAN X (Pascal), pci bus id: 0000:02:00.0, compute capability: 6.1
2023-06-08 11:46:28.463148: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:736] failed to allocate 201.25MiB (211025920 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory


In [4]:
example_dataset = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[4])

cnt = 0
for item in example_dataset:
    cnt += 1 
print(cnt)

100


### Preprocessing and organizing dataset

In [5]:
shuffle_buffer = 100
# todo change??
prefetch_buffer = 10

In [6]:
# Following the preprocessing of your example, update accordingly
def preprocess(dataset):
    def batch_format_fn(element):
        return (tf.reshape(element['pixels'], [-1, 32, 32, 3]), 
                tf.reshape(element['label'], [-1, 1]))

    return dataset.batch(batch_size).map(batch_format_fn)


In [7]:
def poison_dataset(dataset, target_honest, target_mal):
    # parse the dataset
    for batch in dataset:
        # get the labels of each batch and convert to numpy array
        labels = batch['y'].numpy()
        # itterate through each label
        for y in labels:
            # if we find the target label	
            if y == target_honest:
                y = target_mal
        batch['y'] = tf.convert_to_tensor(labels, dtype = tf.float32)
    # return the malicious dataset
    return dataset

In [8]:
## testing the preprocessing function

example_dataset = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[0])
     
preprocessed_example_dataset = preprocess(example_dataset)

# for i in preprocessed_example_dataset:
#     labels = i['y'].numpy()
#     print(labels)
#     labels[0] = 1
#     print(labels)
#     tf_labels = tf.convert_to_tensor(labels, dtype = tf.float32)
#     print(tf_labels)
#     break

# print(preprocessed_example_dataset)

# sample_batch = tf.nest.map_structure(lambda x: x.numpy(),
#                                      next(iter(preprocessed_example_dataset)))



KeyError: in user code:

    File "/tmp/ipykernel_2389283/2574530229.py", line 5, in batch_format_fn  *
        tf.reshape(element['label'], [-1, 1]))

    KeyError: 'pixels'


In [9]:
def make_federated_data(client_data, client_ids):

    fed_data = []
    for id in client_ids:
        preprocessed_dataset = preprocess(client_data.create_tf_dataset_for_client(id))
        # prob = random.random()
        # mal% of the users are malicious
        # if prob < mal_users_percentage:  
        #     preprocessed_dataset = poison_dataset(preprocessed_dataset, target_honest, target_mal)              

        fed_data.append(preprocessed_dataset)

    return fed_data

## Model creation

In [10]:
# def create_keras_model():
#     return tf.keras.models.Sequential([
#         tf.keras.layers.InputLayer(input_shape = (784,)),
#         tf.keras.layers.Dense(10, kernel_initializer = 'zeros'),
#         tf.keras.layers.Softmax(),
#     ])


def create_keras_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(32, 32, 3)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(100, activation='softmax')
    ])


In [11]:
def mnist_model():
    keras_model = create_keras_model()
    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec = preprocessed_example_dataset.element_spec,
        loss = tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics = [tf.keras.metrics.SparseCategoricalAccuracy()])

## Training

In [12]:
training_process = tff.learning.algorithms.build_weighted_fed_avg(
    mnist_model,
    client_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate = client_learning_rate),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate = server_learning_rate))

NameError: name 'preprocessed_example_dataset' is not defined

In [13]:
train_state = training_process.initialize()


NameError: name 'training_process' is not defined

In [14]:
# build the process to have the model's architecture
evaluation_process = tff.learning.algorithms.build_fed_eval(mnist_model)

# initialize the state of the evaluation
evaluation_state = evaluation_process.initialize()
sample_test_clients = emnist_test.client_ids[0:n_test_clients]
# test the model with the test data
# question: selection of clients during training??
federated_test_data = make_federated_data(emnist_test, sample_test_clients)

NameError: name 'preprocessed_example_dataset' is not defined

In [15]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

training_loss = []
training_acc = []
eval_loss = []
eval_acc = []

clients = emnist_train.client_ids[0:n_clients]

#note: slow to converge with random clients, makes sense
federated_train_data = make_federated_data(emnist_train, clients)

for epoch in range(1, n_train_epochs):

    print('Epoch: {:2d}'.format(epoch))
    #client selection, random, chosen from the first 100 clients
    # clients = random.sample(emnist_train.client_ids[0:100], n_clients)

    # run a next on the training process to train the model
    result = training_process.next(train_state, federated_train_data)
    # update the model's state and get access to the metrics
    train_state = result.state
    train_metrics = result.metrics
    # print the training metrics
    training_acc.append(train_metrics['client_work']['train']['sparse_categorical_accuracy'])
    training_loss.append(train_metrics['client_work']['train']['loss'])

    print('Training accuracy: {:.3f}, Training loss: {:.3f}'.format(training_acc[-1], training_loss[-1]))

    # evaluate the model with test data

    # get weights from the trainged model
    model_weights = training_process.get_model_weights(train_state)
    # update the evaluation state with them
    evaluation_state = evaluation_process.set_model_weights(evaluation_state, model_weights)
    # run a next() to evaluate the model
    evaluation_output = evaluation_process.next(evaluation_state, federated_test_data)

    # get access to the evaluation metrics
    eval_metrics = evaluation_output.metrics['client_work']['eval']['total_rounds_metrics']

    eval_acc.append(eval_metrics['sparse_categorical_accuracy'])
    eval_loss.append(eval_metrics['loss'])
    # print the training metrics
    print('Testing accuracy: {:.3f}, Testing loss: {:.3f}\n\n'.format(eval_acc[-1], eval_loss[-1]))

KeyError: in user code:

    File "/tmp/ipykernel_2389283/2574530229.py", line 5, in batch_format_fn  *
        tf.reshape(element['label'], [-1, 1]))

    KeyError: 'pixels'


In [16]:
emnist_train
# clients = emnist_train.client_ids[0:n_clients]
# print(clients)

In [17]:
print(evaluation_output.metrics)

NameError: name 'evaluation_output' is not defined

In [18]:
epochs = [x for x in range(1, n_train_epochs)]
plt.plot(epochs, training_acc)
plt.plot(epochs, eval_acc)
plt.show()

ValueError: x and y must have same first dimension, but have shapes (49,) and (0,)

In [19]:
epochs = [x for x in range(1, n_train_epochs)]
plt.plot(epochs, training_loss)
plt.plot(epochs, eval_loss)
plt.show()

ValueError: x and y must have same first dimension, but have shapes (49,) and (0,)